# **Лабораторна робота 6: Пошук аномалій та вирішення задачі *anomaly detection* за допомогою бібліотек `scikit-learn`та `PyTorch`**
**Всі завдання виконуються індивідуально. Використання запозиченого коду буде оцінюватись в 0 балів.**

**Лабораторні роботи де в коді буде використаня КИРИЛИЦІ будуть оцінюватись в 20 балів.**

### Мета роботи:
Ознайомитися з основними методами виявлення аномалій, навчитися використовувати бібліотеки `scikit-learn` та `PyTorch` для реалізації алгоритмів пошуку аномалій, проаналізувати ефективність різних методів на реальних наборах даних з Kaggle.


### Опис завдання:

1. **Постановка задачі**:
   Використовуючи один із доступних наборів даних Kaggle (наприклад, *Credit Card Fraud Detection*, *Network Intrusion*, або інші), вам потрібно розв'язати задачу виявлення аномалій. Основна мета — ідентифікувати аномальні записи серед нормальних. Вибраний набір даних повинен містити мітки аномалій для перевірки результатів.

2. **Етапи виконання завдання**:
   - Завантажте та підготуйте набір даних.
   - Проведіть попередню обробку даних (масштабування, заповнення пропущених значень, видалення нерелевантних ознак).
   - Використайте різні методи виявлення аномалій:
     - **Методи з бібліотеки scikit-learn**:
       - Isolation Forest
       - One-Class SVM
       - Local Outlier Factor (LOF)
     - **Методи з використанням PyTorch**:
       - Автоенкодери для виявлення аномалій.
   - Порівняйте отримані результати, обчисліть метрики якості (Precision, Recall, F1-Score).
   - Оцініть, який метод найкраще підходить для вирішення задачі на вашому наборі даних.

### Покрокова інструкція

1. **Підготовка середовища**:
   - Встановіть необхідні бібліотеки:
     ```
     pip install scikit-learn torch pandas numpy matplotlib
     ```

2. **Вибір набору даних з Kaggle**:
   Зареєструйтесь на Kaggle та оберіть один із наборів даних для виявлення аномалій. Наприклад:
   - [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud)
   - [Network Intrusion Detection](https://www.kaggle.com/xyuanh/benchmarking-datasets)

3. **Попередня обробка даних**:
   - Завантажте дані та проведіть їхню початкову обробку.
   - Масштабуйте ознаки за допомогою `StandardScaler` або `MinMaxScaler`.
   - Розділіть дані на навчальну і тестову вибірки.

4. **Методи з бібліотеки `scikit-learn`**:

   - **Isolation Forest**:
     ```
     from sklearn.ensemble import IsolationForest
     ```

   - **One-Class SVM**:
     ```
     from sklearn.svm import OneClassSVM
     ```

   - **Local Outlier Factor**:
     ```
     from sklearn.neighbors import LocalOutlierFactor
     ```

5. **Методи на основі нейронних мереж (PyTorch)**:

   Використайте автоенкодер для пошуку аномалій. Побудуйте нейронну мережу з енкодером і декодером. Під час навчання порівняйте відновлені дані з вхідними та обчисліть помилку. Записи з великою помилкою можуть бути аномаліями.

   - **Реалізація автоенкодера**:
     ```
     import torch
     import torch.nn as nn
     import torch.optim as optim
     ```

6. **Оцінка результатів**:
   Використовуйте метрики оцінки якості:
   - `Precision`, `Recall`, `F1-score`
   ```
   from sklearn.metrics import classification_report
   ```

7. **Звіт**:
   - Поясніть, який метод дав найкращі результати.
   - Проаналізуйте, чому деякі методи працюють краще на вашому наборі даних.
   - Оцініть можливості використання глибоких нейронних мереж (автоенкодерів) для вирішення задачі.


### Результати, які необхідно надати:
1. Код рішення у вигляді Jupyter Notebook з аналізом результатів та поясненнями.


### Дедлайн:
[23 жовтня 23:59]


### Корисні ресурси:
- [Документація PyTorch](https://pytorch.org/docs/stable/index.html)
- [Документація scikit-learn](https://scikit-learn.org/stable/documentation.html)
- [Kaggle Datasets](https://www.kaggle.com/datasets)

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [10]:
dataset = pd.read_csv('creditcard.csv')

print(dataset.head())
print(dataset.info())

scaler = StandardScaler()
dataset[['scaled_amount', 'scaled_time']] = scaler.fit_transform(dataset[['Amount', 'Time']])
dataset.drop(['Amount', 'Time'], axis=1, inplace=True)

features = dataset.drop('Class', axis=1)
labels = dataset['Class']

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42, stratify=labels
)

   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [11]:
anomaly_detector = IsolationForest(contamination=0.001, random_state=42)

train_predictions = anomaly_detector.fit_predict(X_train)
test_predictions = anomaly_detector.predict(X_test)
test_predictions = [1 if prediction == -1 else 0 for prediction in test_predictions]

print("Isolation Forest Results:")
print(classification_report(y_test, test_predictions))

Isolation Forest Results:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.35      0.24      0.29        98

    accuracy                           1.00     56962
   macro avg       0.68      0.62      0.64     56962
weighted avg       1.00      1.00      1.00     56962



In [16]:
svm_anomaly_detector = OneClassSVM(gamma='auto', nu=0.001)

svm_anomaly_detector.fit(X_train)
svm_test_predictions = svm_anomaly_detector.predict(X_test)
svm_test_predictions = [1 if prediction == -1 else 0 for prediction in svm_test_predictions]

print("One-Class SVM Results:")
print(classification_report(y_test, svm_test_predictions))

One-Class SVM Results:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     56864
           1       0.07      0.41      0.12        98

    accuracy                           0.99     56962
   macro avg       0.53      0.70      0.56     56962
weighted avg       1.00      0.99      0.99     56962



In [12]:
lof_detector = LocalOutlierFactor(n_neighbors=20, contamination=0.001)

lof_predictions = lof_detector.fit_predict(X_test)
lof_predictions = [1 if prediction == -1 else 0 for prediction in lof_predictions]

print("Local Outlier Factor Results:")
print(classification_report(y_test, lof_predictions))

Local Outlier Factor Results:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.04      0.02      0.03        98

    accuracy                           1.00     56962
   macro avg       0.52      0.51      0.51     56962
weighted avg       1.00      1.00      1.00     56962



In [13]:
class AnomalyAutoencoder(nn.Module):
    def __init__(self, input_dim):
        super(AnomalyAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8)
        )
        self.decoder = nn.Sequential(
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

input_features = X_train.shape[1]
autoencoder_model = AnomalyAutoencoder(input_dim=input_features)

loss_function = nn.MSELoss()
optimizer = optim.Adam(autoencoder_model.parameters(), lr=0.001)

train_tensor = torch.FloatTensor(X_train.values)
test_tensor = torch.FloatTensor(X_test.values)

epochs = 20

for epoch in range(epochs):
    autoencoder_model.train()
    optimizer.zero_grad()

    reconstructed = autoencoder_model(train_tensor)
    reconstruction_loss = loss_function(reconstructed, train_tensor)
    reconstruction_loss.backward()
    optimizer.step()

    print(f'Epoch {epoch + 1}/{epochs}, Loss: {reconstruction_loss.item():.4f}')

Epoch 1/20, Loss: 1.0989
Epoch 2/20, Loss: 1.0982
Epoch 3/20, Loss: 1.0975
Epoch 4/20, Loss: 1.0968
Epoch 5/20, Loss: 1.0962
Epoch 6/20, Loss: 1.0955
Epoch 7/20, Loss: 1.0948
Epoch 8/20, Loss: 1.0942
Epoch 9/20, Loss: 1.0935
Epoch 10/20, Loss: 1.0928
Epoch 11/20, Loss: 1.0921
Epoch 12/20, Loss: 1.0914
Epoch 13/20, Loss: 1.0906
Epoch 14/20, Loss: 1.0898
Epoch 15/20, Loss: 1.0890
Epoch 16/20, Loss: 1.0882
Epoch 17/20, Loss: 1.0873
Epoch 18/20, Loss: 1.0864
Epoch 19/20, Loss: 1.0855
Epoch 20/20, Loss: 1.0845


In [14]:
autoencoder_model.eval()

with torch.no_grad():
    test_reconstructions = autoencoder_model(test_tensor)
    reconstruction_losses = torch.mean((test_reconstructions - test_tensor) ** 2, dim=1)

anomaly_threshold = torch.quantile(reconstruction_losses, 0.99).item()
autoencoder_predictions = (reconstruction_losses > anomaly_threshold).int().numpy()

print("Autoencoder Results:")
print(classification_report(y_test, autoencoder_predictions))

Autoencoder Results:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     56864
           1       0.10      0.56      0.16        98

    accuracy                           0.99     56962
   macro avg       0.55      0.78      0.58     56962
weighted avg       1.00      0.99      0.99     56962



In [17]:
precision_values = []
recall_values = []
f1_values = []

detection_methods = ['Isolation Forest', 'One-Class SVM', 'LOF', 'Autoencoder']
method_predictions = [y_test, svm_test_predictions, lof_predictions, autoencoder_predictions]

for predictions in method_predictions:
    precision_values.append(precision_score(y_test, predictions, zero_division=0))
    recall_values.append(recall_score(y_test, predictions, zero_division=0))
    f1_values.append(f1_score(y_test, predictions, zero_division=0))

print("Performance Metrics by Method:")
for index, method in enumerate(detection_methods):
    print(f"{method}: Precision={precision_values[index]:.4f}, Recall={recall_values[index]:.4f}, F1-Score={f1_values[index]:.4f}")

Performance Metrics by Method:
Isolation Forest: Precision=1.0000, Recall=1.0000, F1-Score=1.0000
One-Class SVM: Precision=0.0677, Recall=0.4082, F1-Score=0.1161
LOF: Precision=0.0351, Recall=0.0204, F1-Score=0.0258
Autoencoder: Precision=0.0965, Recall=0.5612, F1-Score=0.1647


### Звіт

1. **Мета:** Проведено аналіз даних із використанням різних алгоритмів виявлення аномалій:
   - Isolation Forest
   - One-Class SVM
   - Local Outlier Factor (LOF)
   - Autoencoder

2. **Ключові кроки:**
   - **Попередня обробка даних:** Масштабування та розділення на тренувальну і тестову вибірки.
   - **Алгоритми:** Навчання та оцінка моделей для виявлення аномалій.
   - **Метрики:** Розраховано Precision, Recall, F1-Score для кожного методу.

3. **Результати:** Метрики продуктивності показали, що кожен метод має свої переваги та недоліки залежно від структури даних і параметрів.

4. **Висновок:** Найкращий підхід варто обирати на основі контексту застосування, зважаючи на вимоги до точності, чутливості та швидкості моделі.